### **프로젝트 개요**
이 프로젝트는 한국어 법령 질의회신 텍스트에서 '질의 ID', '답변 내용' 등 특정 정보를 자동으로 추출하는 개체명 인식(Named Entity Recognition, NER) 모델을 구축하는 과정을 다룹니다. `klue/bert-base` 모델을 기반으로 Hugging Face 라이브러리를 사용하여 모델을 파인튜닝합니다.

### **학습 목표**
1.  **데이터 준비**: 원시 텍스트를 정제하고, 라벨링 도구(Doccano)에 적합한 형태로 가공합니다.
2.  **데이터 라벨링**: Doccano를 사용하여 텍스트에 개체명 태그를 지정합니다.
3.  **고급 기법 적용**: 약 지도 학습, 데이터 증강, 능동 학습 기법을 적용하여 모델 성능을 효율적으로 향상시킵니다.
4.  **모델 학습 및 평가**: 라벨링된 데이터를 BERT 모델 학습에 맞게 변환하고, 모델을 파인튜닝한 후 성능을 평가합니다.

---

## **1. 환경 설정**
필요한 라이브러리를 설치하고 Google Drive를 연동하여 프로젝트 환경을 구성합니다.

In [ ]:
# 1.1. 필요한 라이브러리 설치
!pip install transformers[torch] datasets evaluate seqeval accelerate -q

# 1.2. 라이브러리 임포트
import os
import json
import re
import numpy as np
import torch
import pandas as pd
from google.colab import drive
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, AutoModelForSeq2SeqLM
import evaluate
from tqdm.auto import tqdm

# 1.3. Google Drive 마운트
drive.mount('/content/gdrive')

# 1.4. 프로젝트 경로 설정
project_root = '/content/gdrive/MyDrive/Colab Notebooks/deep-learning-ner-advanced/'
data_dir = os.path.join(project_root, 'data')
model_dir = os.path.join(project_root, 'model')

os.makedirs(data_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"프로젝트 루트: {project_root}")
print(f"데이터 폴더: {data_dir}")
print(f"모델 저장 폴더: {model_dir}")

---

## **2. 데이터 준비 및 전처리**
모델 학습의 기반이 되는 데이터를 준비하는 단계입니다. 원본 텍스트를 정제하고, 라벨링을 위한 형태로 가공합니다.

### **2.1. 원본 텍스트 파일 준비**
프로젝트에 사용할 원본 텍스트 파일(`dataset.txt`)과, 이 파일을 간단히 정제한 `dataset_cleaned.txt` 파일이 `data` 폴더에 준비되어 있다고 가정합니다.

`dataset_cleaned.txt`는 각 질의-회신 쌍이 두 개의 개행 문자(`\n\n`)로 구분된 파일입니다.

### **(New) 2.2. 약 지도 학습 (Weak Supervision)으로 라벨링 가속화** 🚀

수천 개의 데이터를 처음부터 수동으로 라벨링하는 것은 매우 힘든 작업입니다. **약 지도 학습**은 정규 표현식(Regex)과 같은 간단한 **규칙(Heuristics)을 이용해 대량의 데이터에 자동으로 라벨을 부여**하는 기법입니다.

이렇게 생성된 '부정확하지만 양이 많은' 라벨 데이터는 두 가지 방식으로 활용할 수 있습니다.
1.  **초벌 라벨링 데이터로 사용**: 자동 생성된 `weakly_labeled.jsonl` 파일을 Doccano에 임포트하면, 사람은 처음부터 라벨링하는 대신 **틀린 부분만 빠르게 수정**하면 됩니다. 이를 통해 라벨링 시간을 획기적으로 단축할 수 있습니다.
2.  **모델 사전 학습**: 양질의 수동 라벨 데이터가 아주 적을 경우, 약 지도 학습 데이터로 모델을 먼저 학습시킨 후, 소량의 고품질 데이터로 추가 파인튜닝하는 데 사용할 수 있습니다.

여기서는 1번 방법을 위해 `weakly_labeled.jsonl` 파일을 생성하는 코드를 실행합니다.

In [ ]:
def create_weak_labels(input_text_path, output_jsonl_path):
    """정규식을 사용해 '질의 ID'와 '답변 ID'를 자동으로 라벨링합니다."""
    try:
        with open(input_text_path, 'r', encoding='utf-8') as f:
            # 각 문서는 두 개의 개행 문자로 구분되어 있다고 가정
            documents = f.read().split('\n\n')
    except FileNotFoundError:
        print(f"오류: '{input_text_path}' 파일을 찾을 수 없습니다. 경로를 확인하세요.")
        return

    # '질의' 또는 '회신' 뒤에 숫자와 점(선택)이 오는 패턴
    question_id_pattern = re.compile(r'(질의\s*\d+\.?)')
    answer_id_pattern = re.compile(r'(회신\s*\d+\.?)')
    
    labeled_docs = []
    for doc_text in documents:
        if not doc_text.strip():
            continue

        spans = []
        # QUESTION_ID 라벨링
        for match in question_id_pattern.finditer(doc_text):
            start, end = match.span()
            spans.append([start, end, "QUESTION_ID"])

        # ANSWER_ID 라벨링
        for match in answer_id_pattern.finditer(doc_text):
            start, end = match.span()
            spans.append([start, end, "ANSWER_ID"])

        # Doccano JSONL 형식
        labeled_docs.append({
            "text": doc_text,
            "labels": spans
        })

    # 파일로 저장
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for doc in labeled_docs:
            f.write(json.dumps(doc, ensure_ascii=False) + '\n')

    print(f"약 지도 학습 완료! {len(labeled_docs)}개의 문서가 '{output_jsonl_path}'에 저장되었습니다.")
    print("이 파일을 Doccano에 임포트하여 라벨을 검토하고 수정하세요.")

# 실행
cleaned_text_path = os.path.join(data_dir, 'dataset_cleaned.txt')
weakly_labeled_path = os.path.join(data_dir, 'weakly_labeled.jsonl')
create_weak_labels(cleaned_text_path, weakly_labeled_path)

### **2.3. Doccano를 이용한 데이터 라벨링**
1.  **Doccano 실행**: 로컬 PC에서 Docker를 이용해 Doccano를 실행합니다.
2.  **데이터 임포트**: 위에서 생성한 `weakly_labeled.jsonl` 파일을 `JSONL` 형식으로 임포트합니다.
3.  **라벨링 검토 및 수정**: 자동 라벨링된 결과를 검토하고, 나머지 `QUESTION_CONTENT`, `ANSWER_CONTENT` 등을 수동으로 라벨링합니다.
4.  **데이터 익스포트**: 라벨링이 완료되면 `JSONL` 형식으로, **'Export only approved documents'를 체크**하여 내보냅니다.
5.  **파일 업로드**: 최종 결과물(예: `final_labeled_data.jsonl`)을 Colab의 `data` 폴더에 업로드합니다.

### **2.4. Colab에서 라벨링 데이터 로드 및 가공**
최종 라벨링된 데이터를 로드하고, `klue/bert-base` 토크나이저가 이해할 수 있는 형태로 변환합니다.

In [ ]:
# 2.4.1. 라벨 및 토크나이저 설정
doccano_labels = [
    "QUESTION_ID", "QUESTION_CONTENT", "ANSWER_ID", 
    "ANSWER_CONTENT", "LAW_NAME", "MISC_HEADER"
]

label_list = ["O"] + [f"{tag}-{label}" for label in doccano_labels for tag in ("B", "I")]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

print("전체 라벨 목록:", label_list)

# 2.4.2. Doccano JSONL 파일 로드
labeled_data_filename = 'final_labeled_data.jsonl' # Doccano에서 최종 익스포트한 파일
labeled_data_filepath = os.path.join(data_dir, labeled_data_filename)

try:
    doccano_data = []
    with open(labeled_data_filepath, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            annotations = item.pop('label', item.pop('labels', []))
            item['labels'] = annotations
            doccano_data.append(item)
            
    raw_dataset = Dataset.from_list(doccano_data)
    print(f"\n로드된 라벨링 데이터 샘플 수: {len(raw_dataset)}")

except FileNotFoundError:
    print(f"'{labeled_data_filepath}' 파일을 찾을 수 없습니다. 파일 이름과 경로를 확인하세요.")
    # 예시를 위해 빈 데이터셋 생성
    raw_dataset = Dataset.from_dict({'text':[], 'labels':[]})

# 2.4.3. 토큰화 및 라벨 정렬 함수
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, max_length=512, return_offsets_mapping=True)

    aligned_labels = []
    for i, doc_labels in enumerate(examples["labels"]):
        offset_mapping = tokenized_inputs["offset_mapping"][i]
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        current_labels = np.full(len(offset_mapping), label_to_id['O'])

        for start, end, label in doc_labels:
            b_label_id = label_to_id.get(f"B-{label}")
            i_label_id = label_to_id.get(f"I-{label}")
            if b_label_id is None: continue

            token_indices = [idx for idx, (o_start, o_end) in enumerate(offset_mapping) if o_start >= start and o_end <= end]
            
            if token_indices:
                current_labels[token_indices[0]] = b_label_id
                for idx in token_indices[1:]:
                    current_labels[idx] = i_label_id

        final_labels = [-100 if word_id is None else label_id for word_id, label_id in zip(word_ids, current_labels)]
        aligned_labels.append(final_labels)

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

---

## **3. 모델 학습**
데이터 준비가 완료되면 모델 학습을 시작합니다. 데이터 증강을 통해 학습 데이터의 양을 늘려 모델의 일반화 성능을 향상시킬 수 있습니다.

### **(New) 3.1. 데이터 증강 (Data Augmentation)으로 학습 데이터 확장** 🪄

데이터 증강은 **기존 데이터를 약간 변형하여 새로운 학습 데이터를 생성**하는 기법입니다. 데이터 양이 부족할 때 모델이 다양한 패턴을 학습하게 하여 과적합을 방지하고 성능을 높이는 데 효과적입니다. 

여기서는 **역번역(Back-translation)**을 사용합니다.
1.  원본 한국어 문장을 영어로 번역합니다.
2.  번역된 영어 문장을 다시 한국어로 번역합니다.

이 과정을 거치면 원본과 의미는 같지만 단어나 문장 구조가 미묘하게 다른 새로운 문장을 얻을 수 있습니다. 여기서는 Hugging Face에 공개된 번역 모델(`Helsinki-NLP/opus-mt-ko-en`, `Helsinki-NLP/opus-mt-en-ko`)을 사용합니다.

In [ ]:
# 3.1.1. 역번역을 위한 모델 및 토크나이저 로드
en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
ko_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ko")
ko_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ko")

def back_translate(text):
    """텍스트를 영어로 번역한 뒤 다시 한국어로 번역합니다."""
    # 한국어 -> 영어
    en_tokenized = en_tokenizer(text, return_tensors="pt", truncation=True)
    en_outputs = en_model.generate(**en_tokenized)
    en_text = en_tokenizer.decode(en_outputs[0], skip_special_tokens=True)
    
    # 영어 -> 한국어
    ko_tokenized = ko_tokenizer(en_text, return_tensors="pt", truncation=True)
    ko_outputs = ko_model.generate(**ko_tokenized)
    ko_text = ko_tokenizer.decode(ko_outputs[0], skip_special_tokens=True)
    
    return ko_text

# 3.1.2. 데이터 증강 함수 (주의: 시간이 다소 소요될 수 있습니다)
def augment_data(dataset):
    augmented_texts = []
    print("데이터 증강(역번역)을 시작합니다...")
    for example in tqdm(dataset):
        # 원본 텍스트는 그대로 유지하고, 역번역된 텍스트를 추가
        original_text = example['text']
        # 역번역 실행 (API 호출과 유사하므로 시간이 걸림)
        # translated_text = back_translate(original_text)
        
        # NOTE: 역번역은 시간이 매우 오래 걸리므로, 여기서는 간단한 변형으로 대체 시연합니다.
        # 실제 사용 시에는 위의 back_translate 함수를 사용하세요.
        # 예시: 문장 끝에 공백과 마침표를 추가하는 간단한 증강
        augmented_text = original_text + " ."

        # 증강된 데이터 추가
        # 증강 후에는 라벨의 offset이 변경되므로, 라벨을 'None'으로 처리하고
        # 나중에 재라벨링하거나, offset을 재계산하는 과정이 필요합니다.
        # 여기서는 가장 간단한 형태로, 텍스트만 증강합니다.
        # 실제 프로젝트에서는 증강된 텍스트에 맞게 라벨을 재조정해야 합니다.
        augmented_texts.append({'text': augmented_text, 'labels': example['labels']})
    
    # 기존 데이터와 증강 데이터를 합침
    augmented_dataset = Dataset.from_list(dataset.to_list() + augmented_texts)
    return augmented_dataset

# 3.1.3. 데이터셋 처리 및 증강 적용
if 'raw_dataset' in locals() and len(raw_dataset) > 0:
    # 데이터 증강 적용 (학습 데이터에만 적용)
    # augmented_train_dataset = augment_data(raw_dataset)
    # print(f"증강 후 전체 데이터 수: {len(augmented_train_dataset)}")
    # processed_dataset = augmented_train_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=augmented_train_dataset.column_names)

    # 증강 없이 진행 (시간 관계상)
    print("데이터 증강은 시간이 오래 걸리므로 이 노트북에서는 건너뛰고 진행합니다.")
    processed_dataset = raw_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=raw_dataset.column_names)
    
    # 학습용/평가용 데이터셋 분할
    train_test_split = processed_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']

    print("\n데이터 전처리 및 분할 완료:")
    print(f"학습 데이터셋 샘플 수: {len(train_dataset)}")
    print(f"평가 데이터셋 샘플 수: {len(eval_dataset)}")

### **3.2. 모델 학습 실행**

In [ ]:
# 3.2.1. 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base",
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

# 3.2.2. 성능 지표 정의
seqeval_metric = evaluate.load("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[id_to_label[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[id_to_label[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = seqeval_metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

# 3.2.3. 학습 인자 설정
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8, # 메모리 부족 시 4, 8 등으로 조절
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to="none",
)

# 3.2.4. Trainer 설정 및 학습 시작
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n모델 학습을 시작합니다...")
if len(train_dataset) > 0:
    trainer.train()
    trainer.save_model(os.path.join(model_dir, "best_model"))
    print(f"\n학습 완료! 최적 모델이 '{os.path.join(model_dir, 'best_model')}'에 저장되었습니다.")
else:
    print("학습 데이터셋이 비어 있어 학습을 건너뜁니다.")

---
## **4. 모델 추론 (Inference)**
학습된 모델을 사용하여 새로운 텍스트에서 개체명을 추출하는 테스트를 수행합니다.

In [ ]:
# 4.1. 저장된 모델 로드
model_path = os.path.join(model_dir, "best_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
try:
    inference_tokenizer = AutoTokenizer.from_pretrained(model_path)
    inference_model = AutoModelForTokenClassification.from_pretrained(model_path).to(device)
    print(f"저장된 모델을 로드했습니다. ({model_path})")
except OSError:
    print(f"저장된 모델이 없습니다. '{model_path}' 경로를 확인해주세요. 추론을 건너뜁니다.")
    inference_model = None

# 4.2. 추론 함수
def predict_ner(text, tokenizer, model):
    if model is None: return []
    model.eval()
    tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**tokenized_input)
    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
    predicted_labels = [model.config.id2label[i] for i in predictions[0].cpu().numpy()]
    
    results = []
    current_entity_tokens = []
    current_entity_label = None
    for token, label in zip(tokens, predicted_labels):
        if token in tokenizer.all_special_tokens: continue
        if label.startswith("B-"):
            if current_entity_tokens: results.append({"entity": tokenizer.convert_tokens_to_string(current_entity_tokens), "label": current_entity_label})
            current_entity_tokens = [token]
            current_entity_label = label[2:]
        elif label.startswith("I-") and current_entity_label == label[2:]:
            current_entity_tokens.append(token)
        else:
            if current_entity_tokens: results.append({"entity": tokenizer.convert_tokens_to_string(current_entity_tokens), "label": current_entity_label})
            current_entity_tokens = []
            current_entity_label = None
    if current_entity_tokens: results.append({"entity": tokenizer.convert_tokens_to_string(current_entity_tokens), "label": current_entity_label})
    return results

# 4.3. 테스트 실행
test_text = "질의 1 연면적 450㎡인 특정소방대상물에 최초 건축물 사용승인시에 비상경보설비 설치가 되지 않은 경우 건축허가일과 사용승인일 중 소방시설설치기준 적용일은? 회신 1 건축물 등의 신축ㆍ증축ㆍ개축ㆍ재축ㆍ이전ㆍ용도변경 또는 대수선의 허가 ㆍ협의 및 사용승인의 권한이 있는 행정기관은 소방시설법 제7조제1항에 따라 소재지를 관할하는 소방본부장이나 소방서장의 동의를 받아야 합니다."
predicted_entities = predict_ner(test_text, inference_tokenizer, inference_model)

print("\n--- 추론 결과 ---")
for entity in predicted_entities:
    print(f"- {entity['entity']} ({entity['label']})")

---
## **(New) 5. 능동 학습 (Active Learning)으로 효율적인 라벨링** 🧠

**능동 학습**은 모델 개선을 위해 **어떤 데이터를 라벨링하는 것이 가장 효율적일지 모델 스스로 판단**하게 하는 기법입니다. 수천 개의 unlabeled 데이터 중에서 모델의 성능 향상에 가장 도움이 될 만한 데이터를 골라 사람에게 라벨링을 요청함으로써, 최소한의 노력으로 최대의 성능 향상을 꾀할 수 있습니다.

#### **Active Learning 순환 과정**
1.  소량의 초기 데이터(`seed data`)로 1차 모델을 학습합니다.
2.  학습된 모델을 사용하여 라벨이 없는 대규모 데이터(`unlabeled pool`)에 대해 예측을 수행합니다.
3.  모델이 예측을 가장 **"어려워하는"**, 즉 **불확실성(Uncertainty)이 높은** 데이터를 상위 N개 선택합니다.
4.  선택된 데이터를 사람이 라벨링합니다 (Doccano 사용).
5.  새롭게 라벨링된 데이터를 기존 학습 데이터에 추가하여 모델을 재학습합니다.
6.  원하는 성능에 도달할 때까지 2~5번 과정을 반복합니다.

여기서는 모델 예측의 **엔트로피(Entropy)**를 불확실성의 척도로 사용하여, 라벨링이 필요한 데이터를 찾는 방법을 구현합니다.

In [ ]:
def get_uncertainty_scores(texts, model, tokenizer):
    """텍스트 목록을 입력받아 각 텍스트의 불확실성(평균 엔트로피) 점수를 계산합니다."""
    if model is None: return [], []
    model.eval()
    scores = []
    
    print("라벨 없는 데이터에 대해 불확실성 점수를 계산합니다...")
    for text in tqdm(texts):
        tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
        with torch.no_grad():
            logits = model(**tokenized_input).logits
        
        # 로짓을 확률로 변환
        probabilities = torch.softmax(logits, dim=-1)
        # 각 토큰의 엔트로피 계산
        entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-9), dim=-1)
        # 문장의 평균 엔트로피를 불확실성 점수로 사용 (특수 토큰 제외)
        input_ids = tokenized_input.input_ids[0]
        valid_token_entropy = [e for e, i in zip(entropy[0], input_ids) if i not in tokenizer.all_special_ids]
        
        if valid_token_entropy:
            scores.append(torch.mean(torch.stack(valid_token_entropy)).item())
        else:
            scores.append(0)
            
    return scores

# unlabeled 데이터 풀이 있다고 가정 (예: dataset_cleaned.txt에서 일부 사용)
unlabeled_pool_path = os.path.join(data_dir, 'dataset_cleaned.txt')
try:
    with open(unlabeled_pool_path, 'r', encoding='utf-8') as f:
        unlabeled_texts = f.read().split('\n\n')
except FileNotFoundError:
    unlabeled_texts = []

if unlabeled_texts and inference_model is not None:
    # 불확실성 점수 계산
    uncertainty_scores = get_uncertainty_scores(unlabeled_texts, inference_model, inference_tokenizer)
    
    # 점수를 기준으로 내림차순 정렬
    sorted_indices = np.argsort(uncertainty_scores)[::-1]
    
    # 상위 10개 데이터를 라벨링 대상으로 선정
    num_to_label = 10
    print(f"\n가장 불확실성이 높은 상위 {num_to_label}개의 문서를 선정했습니다. (라벨링 대상)")
    
    to_label_texts = []
    for i in sorted_indices[:num_to_label]:
        print(f"- Score: {uncertainty_scores[i]:.4f}, Text: {unlabeled_texts[i][:80]}...")
        to_label_texts.append(unlabeled_texts[i])

    # 선정된 데이터를 파일로 저장하여 Doccano에 임포트할 수 있도록 준비
    active_learning_output_path = os.path.join(data_dir, 'needs_labeling.txt')
    with open(active_learning_output_path, 'w', encoding='utf-8') as f:
        f.write('\n\n'.join(to_label_texts))
    print(f"\n라벨링 대상 파일이 '{active_learning_output_path}'에 저장되었습니다.")
else:
    print("\nUnlabeled 데이터가 없거나 모델이 학습되지 않아 Active Learning을 건너뜁니다.")